In [317]:
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle
import numpy as np
import time

In [318]:
#whether to use cuda
use_cuda = True

In [342]:
#load the training/test set
training_set = np.load("data/train_imgs.npy",allow_pickle=True)
training_lbl = np.load("data/train_coords.npy",allow_pickle=True)
training_lbl = training_lbl.reshape(training_lbl.shape[0],6)

print(training_set.shape)
print(training_lbl.shape)

(1000, 64, 64)
(1000, 6)


In [343]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 11.3
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce GTX 1070


In [344]:
#conver the data to tensor and put it in gpu
if (use_cuda):
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



training_set = torch.tensor(training_set)
training_lbl = torch.tensor(training_lbl)

training_set = training_set.to(device)
training_lbl = training_lbl.to(device)

print(training_set.shape)

training_set = training_set.float()
training_lbl = training_lbl.float()

torch.Size([1000, 64, 64])


In [345]:
training_set.device
training_lbl.device

device(type='cuda', index=0)

In [385]:
#Define the model

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(3, 3, 5)

        self.fc1 = nn.Linear(507, 16 * 16)
        self.fc2 = nn.Linear(16*16, 8*8)
        self.fc3 = nn.Linear(8*8, 4*4)
        self.fc4 = nn.Linear(4*4, 6)

        self.do = torch.nn.Dropout(p = 0.5)

    def forward(self, x):


        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1) # flatten all dimensions except batch

        x = self.do(x)



        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x


net = Net()
net.to(device)

net.train()

Net(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 3, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=507, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=6, bias=True)
  (do): Dropout(p=0.5, inplace=False)
)

In [386]:
def loss_euc(output,lbl):
    align0 = output.clone()
    align1 = output.clone()
    align1[:,2] = output[:,4]
    align1[:,3] = output[:,5]
    align1[:,4] = output[:,2]
    align1[:,5] = output[:,3]

    align2 = output.clone()
    align2[:,0] = output[:,2]
    align2[:,1] = output[:,3]
    align2[:,2] = output[:,0]
    align2[:,3] = output[:,1]

    align3 = align2.clone()
    align3[:,2] = align2[:,4]
    align3[:,3] = align2[:,5]
    align3[:,4] = align2[:,2]
    align3[:,5] = align2[:,3]

    align4 = output.clone()
    align4[:,0] = output[:,4]
    align4[:,1] = output[:,5]
    align4[:,4] = output[:,0]
    align4[:,5] = output[:,1]

    align5 = align4.clone()
    align5[:,2] = align4[:,4]
    align5[:,3] = align4[:,5]
    align5[:,4] = align4[:,2]
    align5[:,5] = align4[:,3]

    out0 = (align0 - training_lbl)**2
    out1 = (align1 - training_lbl)**2
    out2 = (align2 - training_lbl)**2
    out3 = (align3 - training_lbl)**2
    out4 = (align4 - training_lbl)**2
    out5 = (align5 - training_lbl)**2

    sum0 = torch.sum(out0,dim = 1)
    sum1 = torch.sum(out1,dim = 1)
    sum2 = torch.sum(out2,dim = 1)
    sum3 = torch.sum(out3,dim = 1)
    sum4 = torch.sum(out4,dim = 1)
    sum5 = torch.sum(out5,dim = 1)

    sums = torch.stack([sum0,sum1,sum2,sum3,sum4,sum5]).T

    mins = torch.min(sums,dim = 1).values

    #return torch.mean((output - lbl)**2)
    return torch.mean(mins)

In [402]:
#Define a loss function
criterion = nn.MSELoss()

#Train the model with a high learning rate first
opt = torch.optim.Adam(net.parameters(), 0.00001)
for ii in range(10000):
    t = time.time()

    opt.zero_grad()
    outputs = net(training_set.reshape(1000,1,64,64))

    loss =loss_euc(outputs,training_lbl)


    #loss = criterion(outputs,training_lbl)
    loss.backward()
    opt.step()

    if (ii % 100== 0):
        print(loss,time.time() - t)



tensor(6.0854, device='cuda:0', grad_fn=<MeanBackward0>) 0.009971380233764648
tensor(6.0172, device='cuda:0', grad_fn=<MeanBackward0>) 0.03093123435974121
tensor(6.4013, device='cuda:0', grad_fn=<MeanBackward0>) 0.030155658721923828
tensor(5.8230, device='cuda:0', grad_fn=<MeanBackward0>) 0.046540260314941406
tensor(5.8184, device='cuda:0', grad_fn=<MeanBackward0>) 0.030847787857055664
tensor(5.6626, device='cuda:0', grad_fn=<MeanBackward0>) 0.031707763671875
tensor(6.3300, device='cuda:0', grad_fn=<MeanBackward0>) 0.03137969970703125
tensor(6.0154, device='cuda:0', grad_fn=<MeanBackward0>) 0.031463623046875
tensor(6.0127, device='cuda:0', grad_fn=<MeanBackward0>) 0.04265189170837402
tensor(5.9077, device='cuda:0', grad_fn=<MeanBackward0>) 0.029107332229614258
tensor(6.3667, device='cuda:0', grad_fn=<MeanBackward0>) 0.04294395446777344
tensor(6.1835, device='cuda:0', grad_fn=<MeanBackward0>) 0.027894020080566406
tensor(5.9421, device='cuda:0', grad_fn=<MeanBackward0>) 0.030926704406738

KeyboardInterrupt: 

In [403]:
torch.save(net.state_dict(), "model_checkpoints/single_triangle_4_10.pt")